# Bat gut microbio metagenomic analysis

TJ Rogers

In [2]:
import pandas as pd

## 8-Oct-2024

* To do:
  - [X] Find good host sequences so that I can remove host reads from the metagenomes.
  - [X] Run snakemake workflow to trim reads and retrieve host reference sequences.
  - [X] Create script to remove host reads from the fastq files.
  - [X] Update the main `README.md` file
  - [ ] Check the reference file to see if they all end in "_mPhyDis1.pri.v3_genomic.fna". If not, need to figure out how to reference them in the params section of the rule remove_host_reads in `readQC.smk`*

### Find good host sequences so that I can remove host reads from the metagenomes.

Below, I have listed the sample name (host) along with the species name and the accession number for the reference seuquence:

In [3]:
metadata={'sample': ['NBS1051F-30-542494038','NBS1051F','NBS1079E-30-542494038','NBS1079E-30-550131958','080613-8-pd','080813-19-pd','080813-7-mc','080913-2-mc','080913-7-ph'],
          'species': ['Pteronotus parnellii','Pteronotus parnellii','Myotis elegans', 'Myotis elegans','Phyllostomus discolor','Phyllostomus discolor','Mimon crenulatum','Mimon crenulatum','Phyllostomus hastatus'],
          'accession': ['GCA_036768555.1','GCA_036768555.1','NaN','NaN','GCA_004126475.3','GCA_004126475.3','NaN','NaN','GCA_019186645.2']}

# Create DataFrame
metadata = pd.DataFrame(metadata)
print(metadata)

                  sample                species        accession
0  NBS1051F-30-542494038   Pteronotus parnellii  GCA_036768555.1
1               NBS1051F   Pteronotus parnellii  GCA_036768555.1
2  NBS1079E-30-542494038         Myotis elegans              NaN
3  NBS1079E-30-550131958         Myotis elegans              NaN
4            080613-8-pd  Phyllostomus discolor  GCA_004126475.3
5           080813-19-pd  Phyllostomus discolor  GCA_004126475.3
6            080813-7-mc       Mimon crenulatum              NaN
7            080913-2-mc       Mimon crenulatum              NaN
8            080913-7-ph  Phyllostomus hastatus  GCA_019186645.2


* I already have the script written to download sequences NCBI, but the genome of some host have not been sequenced and are not found on any data base. For instance, neither Myotis elegans nor Mimon crenulatum have had their genomes sequences.
  * However, according to Hurtado and D'Elia-2018, Mimon crenulatum is not an actual species. It is actually a clade that is composed of multiple species. One such species is M. crenulatum keenani. I found that the database 'bat1k' has the genome of M. crenulatum keenani sequenced. So I will retrieve the sequence from there. 
  * I am currently waiting on bat1k to approve my membership so I can download the sequence data I need.

### Run snakemake workflow to trim reads and retrieve host reference sequences.


* JobID: 11161798:
  * Apparently this version of snakemake can not take in bash script in the arguement "script:". I got the following error:
    * "Unsupported script: Expecting either Python (.py), R (.R), RMarkdown (.Rmd) or Julia (.jl) script."

  * I added the script path to "params" and changed the argument "script" to "shell". Going to try again.

* JobID: 11161965
  * I made a mistake by giving the same output read file for the fr1 and fr2 slot in the in the shell section of the clean_raw_reads rule in the file readQC.smk. 
  * Made the required change and running again.

* JobID: 11162659
  * For most of the samples, the adapter and phix removal worked. However, for some reason the temp fq file for sample 080913-2-mc isnt being created correctly or some other error is happening with it as snakemake is removing it and faulting out.
  * I am troubleshooting now:
    * Going to set up the rule to where it will create a log so I can use that to hopefully troubleshoot this sample.

* JobID: 11163418
  * I noticed that I was having the script place the trimmed reads in the 'tmp' directory. I have changed that in the script. 
  * Once this job has stopped I need to:
    - [x] mv the trimmed fq files to the clean_reads directory.
  * This job failed as well for the same reason as the previous job. It said that line 23 in `trimming.sh` was not a command. Problem is I had already modified that script and was unable to figure out what line it was talking about. I am trying to run again to see what will happen.

* JobID: 11163492
  * Job completed without any issues as of 09-Oct-2024

### Create script to remove host reads from the fastq files.

* While I am currently creating this script, it will not be able to be tested until bat1k has given me access to their data base.
  * Script name: `host_read_removal.sh`
  * path: `workflow/scripts/`

## 9-Oct-2024

* To do:
  - [X] Check email to see if Bat1k has granted you access to download their sequences.
  - [X] Find missing host reference sequences or good alternatives. 
  - [X] Check status of JobID 11163492 and update on previous day's notes.
  - [ ] Read lit papers in reference to bats, bat microbiomes, and HiC ananysis.
  - [ ] Write script for assembling reads.
  - [ ] Make needed adjustments to the `host_read_removal.sh`.
  - [ ] Update the main `README.md` file
  - [ ] Check the reference file to see if they all end in "_mPhyDis1.pri.v3_genomic.fna". If not, need to figure out how to reference them in the params section of the rule remove_host_reads in `readQC.smk`

### Check email to see if Bat1k has granted you access to download their sequences.

* Bat1k has not sent me an update as of yet. I will wait until this afternoon. If they have not gotten a hold of me by that time, I will reach out to them again.
  * I found replacement reference sequences on NCBI and no longer need access to Bat1k.

### Find missing host reference sequences or good alternatives. 


* Myotis elegans does not have a whole genome sequences. However, Stadelmann et al. 2007 and Moratell et al. 2013 suggest that M. elegans closest relatives are (in order of closest to least) M. simus, M. riparius, M. ruber, and M. keaysi. This gives me some options to look for. I will check the NCBI data base to see if there are seuqences for any of these to use them as a reference genome. If not, I will work my way further down the tree until i get a hit.
  * None of the above listed options have had their genome sequenced. Howerever, I did find two genomes from members within the same clade: M. yumanesis and M. vivesi. M. vivesi is larger and eats fish as well as insects, whereas M. yumanesis seems to be more like M. elegans in size and diet. So I am going to use M. yumanesis as a stand in.
* Mimon crenulatum (Gardnerycteris crenulatum) does not have a whole genome sequenced. However, it turns out that Phyllostomus discolor is a close relative of M. crenulatum (Hurtado et. al, 2018). So I will use it as a reference genome.

* Below, I have updated the metatable to reflect new additions to the reference sequences:

In [5]:
metadata2={'sample': ['NBS1051F-30-542494038','NBS1051F','NBS1079E-30-542494038','NBS1079E-30-550131958','080613-8-pd','080813-19-pd','080813-7-mc','080913-2-mc','080913-7-ph'],
          'species': ['Pteronotus parnellii','Pteronotus parnellii','Myotis elegans', 'Myotis elegans','Phyllostomus discolor','Phyllostomus discolor','Mimon crenulatum','Mimon crenulatum','Phyllostomus hastatus'],
          'accession': ['GCA_036768555.1','GCA_036768555.1','GCA_028538775.1','GCA_028538775.1','GCA_004126475.3','GCA_004126475.3','GCA_004126475.3','GCA_004126475.3','GCA_019186645.2'],
          'substitute_species': ['none','none','Myotis yumanensis','Myotis yumanensis','none','none','Phyllostomus discolor','Phyllostomus discolor','none']}

# Create DataFrame
metadata2 = pd.DataFrame(metadata2)
metadata2

,sample,species,accession,substitute_species
0,NBS1051F-30-542494038,Pteronotus parnellii,GCA_036768555.1,none
1,NBS1051F,Pteronotus parnellii,GCA_036768555.1,none
2,NBS1079E-30-542494038,Myotis elegans,GCA_028538775.1,Myotis yumanensis
3,NBS1079E-30-550131958,Myotis elegans,GCA_028538775.1,Myotis yumanensis
4,080613-8-pd,Phyllostomus discolor,GCA_004126475.3,none
5,080813-19-pd,Phyllostomus discolor,GCA_004126475.3,none
6,080813-7-mc,Mimon crenulatum,GCA_004126475.3,Phyllostomus discolor
7,080913-2-mc,Mimon crenulatum,GCA_004126475.3,Phyllostomus discolor
8,080913-7-ph,Phyllostomus hastatus,GCA_019186645.2,none


### Check status of JobID 11163492 and update on previous day's notes.

* Job is complete. References have been downloaded. Only issue is that I did not inlcude a command in the script to unzip the .zip file. Updating and running that portion again.

* JobID:11188202
  * This job failed. Not sure what the issue was. But I have split the rule 'get_host_references' into two: 'get_host_references' and 'decompress_references'

* JobID: 11188332
  * Job complete. Now to test if the decompress_references rule works. First, I will need to unzip the ncbi file to see what the common file path is.

* JobID: 11188819 (Testing unzip on ncbi reference file)